<a href="https://colab.research.google.com/github/2003saurabh/deep_learning/blob/main/age_gender_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 94% 313M/331M [00:03<00:00, 144MB/s]
100% 331M/331M [00:03<00:00, 107MB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [5]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [7]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [8]:
len(age)

23708

In [9]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [10]:
df.shape

(23708, 3)

In [11]:
df.head()

,age,gender,img
0,26,1,26_1_3_20170117154752639.jpg.chip.jpg
1,29,0,29_0_3_20170119195114531.jpg.chip.jpg
2,72,0,72_0_0_20170111222541402.jpg.chip.jpg
3,46,0,46_0_4_20170117190308338.jpg.chip.jpg
4,31,0,31_0_1_20170117135443579.jpg.chip.jpg


In [12]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [13]:
train_df.shape

(20000, 3)

In [14]:
test_df.shape

(3708, 3)

In [30]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [62]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],  # Ensure this is correct
    target_size=(200, 200),
    class_mode='raw',  # This ensures outputs are not wrapped in lists
    batch_size=32
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],  # Ensure this is correct
    target_size=(200, 200),
    class_mode='raw',  # This ensures outputs are not wrapped in lists
    batch_size=32
)


Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [52]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [53]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [54]:
resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [55]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [56]:
#model.summary()

In [66]:
model.compile(optimizer='adam',
              loss={'age': 'mae', 'gender': 'binary_crossentropy'},
              metrics={'age': 'mae', 'gender': 'accuracy'},
              loss_weights={'age': 1.0, 'gender': 99.0})  # Ensure values are floats


In [67]:
import tensorflow as tf

tf.config.run_functions_eagerly(True)


In [68]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


625/625 ━━━━━━━━━━━━━━━━━━━━ 440s 695ms/step - age_mae: 17.3030 - loss: 17.3030 - val_age_mae: 16.2789 - val_loss: 16.2789
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 450s 718ms/step - age_mae: 16.5209 - loss: 16.5209 - val_age_mae: 16.2775 - val_loss: 16.2775
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 498s 708ms/step - age_mae: 16.4980 - loss: 16.4980 - val_age_mae: 16.2761 - val_loss: 16.2761
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 489s 691ms/step - age_mae: 16.4829 - loss: 16.4829 - val_age_mae: 16.2765 - val_loss: 16.2765
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 452s 707ms/step - age_mae: 16.3362 - loss: 16.3362 - val_age_mae: 16.2760 - val_loss: 16.2760
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 490s 687ms/step - age_mae: 16.5176 - loss: 16.5176 - val_age_mae: 16.2760 - val_loss: 16.2760
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 431s 686ms/step - age_mae: 16.4446 - loss: 16.4446 - val_age_mae: 16.2763 - val_loss: 16.2763
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 432s 688ms/step - age_mae: 16.364